In [1]:
!pip install requests


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import requests

In [3]:

# Set the base URL for your API
BASE_URL = 'http://127.0.0.1:8001/uploads/file_uploads'

# Define the GUID and file information
guid = 'd1.zip'  # Replace with an actual GUID from your database

# Headers for the requests
headers = {
    'Content-Type': 'application/json',
}


In [27]:

# Function to update the priority
def update_priority(guid, priority):    
    url = f'{BASE_URL}/update_priority/'
    data = {
        'guid': guid,
        'priority': priority,
    }
    response = requests.post(url, json=data, headers=headers)
    print('Update Priority Response:', response.json())


In [5]:

# Function to update the status
def update_status(guid, status):
    url = f'{BASE_URL}/update_status/'
    data = {
        'guid': guid,
        'status': status,
    }
    response = requests.post(url, json=data, headers=headers)
    print('Update Status Response:', response.json())


In [6]:

# Test updating the priority
print("Testing Update Priority API")
update_priority(guid, 1)  # Set priority to 1


Testing Update Priority API
Update Priority Response: {'error': 'File not found'}


In [12]:

# Test pausing the upload
print("\nTesting Pause Upload API")
update_status(guid, 'paused')



Testing Pause Upload API
Update Status Response: {'status': 'status updated'}


In [ ]:

# Test resuming the upload
print("\nTesting Resume Upload API")
update_status(guid, 'resume')



In [ ]:

# # Test canceling the upload
# print("\nTesting Cancel Upload API")
# update_status(guid, 'cancel')

In [10]:
import pickle

serialized_data = b"\x80\x04\x95\xe7\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\tfile_path\x94\x8c\x13/tmp/test/test4.zip\x94\x8c\x0bobject_name\x94\x8c\ttest4.zip\x94\x8c\x04guid\x94h\x04\x8c\x0cinstance_uid\x94\x8c\x11some_instance_uid\x94\x8c\bpriority\x94J}Byf\x8c\x06status\x94\x8c\tuploading\x94\x8c\ncreated_at\x94GA\xd9\x9eP\x9f@\xdc>\x8c\nupdated_at\x94GA\xd9\x9eP\xa8\xd0\xc6}\x8c\ttimestamp\x94GA\xd9\x9eP\x9f@\xdcR\x8c\bprogress\x94G@QJ\n\x9b[\xc6\x9cu."

# Deserialize the data
data = pickle.loads(serialized_data)

# Print the resulting object
print(data)


{'file_path': '/tmp/test/test4.zip', 'object_name': 'test4.zip', 'guid': 'test4.zip', 'instance_uid': 'some_instance_uid', 'priority': 1719222909, 'status': 'uploading', 'created_at': 1719222909.0134425, 'updated_at': 1719222947.2621148, 'timestamp': 1719222909.0134473, 'progress': 69.15689739191515}


In [6]:
import time

In [28]:

guids = ['m1.zip', 'm2.zip','m3.zip','m4.zip','m5.zip','m6.zip','m7.zip','m8.zip','m9.zip']

# Test Case 1: Change priority to the highest
print("Test Case 1: Change priority to the highest")
update_priority(guids[6], 1)
time.sleep(1)


Test Case 1: Change priority to the highest
Update Priority Response: {'error': 'File not found'}


In [12]:

# Test Case 2: Change priority of multiple files
print("Test Case 2: Change priority of multiple files")
update_priority(guids[1], 1)
update_priority(guids[2], 3)
update_priority(guids[3], 4)
time.sleep(1)


Test Case 2: Change priority of multiple files
Update Priority Response: {'status': 'priority updated'}
Update Priority Response: {'status': 'priority updated'}
Update Priority Response: {'status': 'priority updated'}


In [ ]:

# Test Case 3: Change priority to a lower level
print("Test Case 3: Change priority to a lower level")
update_priority(guids[0], 4)
time.sleep(1)

# Test Case 4: Change priority and resume
print("Test Case 4: Change priority and resume")
update_status(guids[1], 'paused')
update_priority(guids[1], 1)
update_status(guids[1], 'resume')
time.sleep(1)

# Test Case 5: Ensure unique priorities
print("Test Case 5: Ensure unique priorities")
update_priority(guids[0], 1)
update_priority(guids[1], 2)
update_priority(guids[2], 3)
update_priority(guids[3], 4)
time.sleep(1)


In [ ]:

# Test Case 6: Swap priorities between files
print("Test Case 6: Swap priorities between files")
update_priority(guids[0], 2)
update_priority(guids[1], 1)
time.sleep(1)


In [ ]:

# Test Case 7: Handle maximum uploads
print("Test Case 7: Handle maximum uploads")
update_priority(guids[0], 1)
update_priority(guids[1], 1)
update_priority(guids[2], 1)
update_priority(guids[3], 1)  # Assuming this is more than MAX_UPLOADS
time.sleep(1)


In [ ]:

# Test Case 8: Change priority of a non-existent file
print("Test Case 8: Change priority of a non-existent file")
update_priority('non_existent_guid', 1)
time.sleep(1)


In [ ]:

# Test Case 9: Change priority with invalid data
print("Test Case 9: Change priority with invalid data")
update_priority(guids[0], 'invalid_priority')
update_priority(guids[0], -1)  # Assuming negative values are invalid
time.sleep(1)


In [ ]:

# Test Case 10: Pause high priority task
print("Test Case 10: Pause high priority task")
update_priority(guids[0], 1)
update_status(guids[1], 'paused')
time.sleep(1)

In [32]:
import hashlib
import boto3

In [30]:
def calculate_sha256(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, 'rb') as f:
        for chunk in iter(lambda: f.read(4096), b''):
            sha256.update(chunk)
    return sha256.hexdigest()



In [37]:
def calculate_s3_sha256( bucket, key):
    s3_client = boto3.client('s3')
    # Get the object and its metadata
    s3_object = s3_client.head_object(Bucket=bucket, Key=key)
    # Get the SHA256 checksum from the metadata
    s3_checksum = s3_object['ETag']
    return s3_checksum

In [41]:
def calculate_s3_sha256( bucket, key):
    s3_client = boto3.client('s3')
    s3_object = s3_client.get_object(Bucket=bucket, Key=key)
    s3_sha256 = hashlib.sha256(s3_object['Body'].read()).hexdigest()
    return s3_sha256


In [43]:
def calculate_s3_sha256( bucket, key):
    s3_client = boto3.client('s3')
    response=s3_client.get_object_attributes(
            Bucket=bucket,
            Key=key,
            ObjectAttributes=['Checksum']
        )
    s3_checksum = response['Checksum']['ChecksumSHA256']
    return s3_checksum

In [45]:
calculate_s3_sha256("cdk-hnb659fds-assets-182426352951-ap-southeast-1","m8.zip")

KeyError: 'Checksum'

In [39]:
calculate_sha256("/tmp/test/m8.zip")

'12dcbded7f1babd8635648cd8d46894bd474077b2f175fdeaa1ee402009dac05'